In [8]:
import os
import pickle
import numpy as np
import pandas as pd
import scipy.stats

import util_localization_psychophysics

tqdm.tqdm.pandas()


In [4]:
"""
Run all sound localization experiments.

`list_regex_dir_model` is a list of model conditions
(different phase locking limits / cochlear models).

Each directory included must contain the model evaluation
output files for each psychophysical stimulus dataset (all
model predictions, output probabilities, and stimulus data).
"""

list_regex_dir_model = [
    "models/sound_localization/IHC3000_delayed_integration/arch??",
    "models/sound_localization/IHC3000/arch??",
    "models/sound_localization/IHC1000/arch??",
    "models/sound_localization/IHC0320/arch??",
    "models/sound_localization/IHC0050/arch??",
    "models/sound_localization/simplified_IHC3000_delayed_integration/arch??",
    "models/sound_localization/simplified_IHC3000/arch??",
    "models/sound_localization/simplified_IHC1000/arch??",
    "models/sound_localization/simplified_IHC0320/arch??",
    "models/sound_localization/simplified_IHC0050/arch??",
]

EXPERIMENT_DATAFRAMES = util_localization_psychophysics.run_localization_experiments(
    list_regex_dir_model,
    workers=60,
    dir_human_data='data/human/sound_localization',
    tag_ckpt='',
    func_label_to_azim_elev=util_localization_psychophysics.label_to_azim_elev,
    key_pred_prob='label_loc_int:probs_out',
    key_pred='label_loc_int:labels_pred',
    key_true='label_loc_int:labels_true',
    n_loc_classes=504,
)


Completed experiment `maa_azimuth` for 10 `models/sound_localization/IHC3000_delayed_integration/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/IHC3000/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/IHC1000/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/IHC0320/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/IHC0050/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/simplified_IHC3000_delayed_integration/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/simplified_IHC3000/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/simplified_IHC1000/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/simplified_IHC0320/arch??` models
Completed experiment `maa_azimuth` for 10 `models/sound_localization/simplified_IHC0050/

In [6]:
"""
Save sound localization experiment data to files.

A dictionary of all experiment results is saved to
a single pickle file. Individual experiment results
are also saved to separate CSV files.
"""

fn_data = 'data/model/sound_localization.pkl'
with open(fn_data, 'wb') as f:
    pickle.dump(EXPERIMENT_DATAFRAMES, f)
print(f"Wrote: {fn_data}")

for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    fn_data_expt = fn_data.replace('.pkl', f'/{tag_expt}.csv')
    EXPERIMENT_DATAFRAMES[tag_expt].to_csv(fn_data_expt, index=False)
    print(f"Wrote: {fn_data_expt}")


Wrote: data/model/sound_localization.pkl
Wrote: data/model/sound_localization/maa_azimuth.csv
Wrote: data/model/sound_localization/itd_threshold.csv
Wrote: data/model/sound_localization/itd_ild_weighting.csv
Wrote: data/model/sound_localization/spectral_smoothing.csv
Wrote: data/model/sound_localization/precedence_effect_localization.csv
Wrote: data/model/sound_localization/new_ears.csv
Wrote: data/model/sound_localization/bandwidth_dependency.csv
Wrote: data/model/sound_localization/mp_spectral_cues.csv
Wrote: data/model/sound_localization/snr_dependency.csv
Wrote: data/model/sound_localization/speech_in_noise_in_reverb.csv


In [7]:
"""
Load all sound localization experiment data.
"""

fn_data = 'data/model/sound_localization.pkl'
with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
print(f"Loaded: {fn_data_expt}")
for tag_expt in EXPERIMENT_DATAFRAMES.keys():
    print(f"|__ {tag_expt}")


Loaded: data/model/sound_localization/speech_in_noise_in_reverb.csv
|__ maa_azimuth
|__ itd_threshold
|__ itd_ild_weighting
|__ spectral_smoothing
|__ precedence_effect_localization
|__ new_ears
|__ bandwidth_dependency
|__ mp_spectral_cues
|__ snr_dependency
|__ speech_in_noise_in_reverb


In [9]:
"""
Calculate human-model comparison metrics for
all sound localization experiments and save
output to a file.
"""

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    'rmse': rmse,
}

df_results = []
for tag_expt in tqdm.tqdm(EXPERIMENT_DATAFRAMES.keys()):
    df = EXPERIMENT_DATAFRAMES[tag_expt]
    df_results_tmp = util_localization_psychophysics.compare_localization_experiment(
        df,
        tag_expt=tag_expt,
        dict_metric_function=dict_metric_function,
        bootstrap_repeats=1000,
        workers=60)
    if df_results_tmp is not None:
        df_results.append(df_results_tmp)
df_results = pd.concat(df_results)

fn_metrics = 'data/model/sound_localization_human_model_comparison_metrics.pkl'
df_results.to_pickle(fn_metrics)
print(f"Wrote: {fn_metrics}")


 10%|██████████████▉                                                                                                                                      | 1/10 [00:18<02:47, 18.59s/it]/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
/om2/user/msaddler/.conda/envs/tf/lib/python3.11/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [02:52<00:00, 17.27s/it]


tag_expt='speech_in_noise_in_reverb'` not recognized --> returning None
Wrote: data/model/sound_localization_human_model_comparison_metrics.pkl


In [ ]:
fn = 'data/human/word_spkr_recognition/word_and_spkr_recognition_with_pitch_altered_speech.pkl'
df = pd.read_pickle(fn)

df_spkr = df[['condition', 'correct_spkr_list', 'f0_shift_in_semitones', 'tag_expt', 'tag_model']]
df_spkr = df_spkr.rename(columns={'correct_spkr_list': 'correct_spkr'})
df_word = df[['condition', 'correct_word_list', 'f0_shift_in_semitones', 'tag_expt', 'tag_model']]
df_word = df_word.rename(columns={'correct_word_list': 'correct_word'})

df_spkr['fn_eval'] = df_spkr['correct_spkr'].map(lambda l: ['spkr_participant_{:04d}'.format(_) for _ in range(len(l))])
df_spkr = df_spkr.explode(['fn_eval', 'correct_spkr'])

df_word['fn_eval'] = df_word['correct_word'].map(lambda l: ['word_participant_{:04d}'.format(_) for _ in range(len(l))])
df_word = df_word.explode(['fn_eval', 'correct_word'])

df = pd.concat([df_spkr, df_word])
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv(fn.replace('.pkl', '.csv'), index=False)

# df.groupby(['condition', 'f0_shift_in_semitones']).agg({
#     'correct_spkr': list,
#     'correct_word': list,
#     'fn_eval': list,
# }).reset_index(drop=False)



In [ ]:
importlib.reload(util_spkr_word_psychophysics)

list_regex_dir_model = [
    'saved_models/augmented_2022JAN/taskSW/IHC3000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/IHC1000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/IHC0320Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/IHC0050Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/spont0_simplified_IHC3000/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/spont0_simplified_IHC1000/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/spont0_simplified_IHC0320/arch0_00??',
    'saved_models/augmented_2022JAN/taskSW/spont0_simplified_IHC0050/arch0_00??',
    'saved_models/augmented_2022JAN/task_S/IHC3000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_S/IHC1000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_S/IHC0320Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_S/IHC0050Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_W/IHC3000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_W/IHC1000Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_W/IHC0320Hz_anf384H160M096L/arch0_00??',
    'saved_models/augmented_2022JAN/task_W/IHC0050Hz_anf384H160M096L/arch0_00??',
]

dict_basename_eval = {
    'kell_like_inharmonic': 'EVAL_word_recognition_human_experiment_v00_inharmonic_foreground60dbspl.json',
    'kell_like': 'EVAL_word_recognition_human_experiment_v00_foreground60dbspl.json',
    'speech_in_synthetic_textures': 'EVAL_word_recognition_speech_in_synthetic_textures.json',
    'pitch_altered': 'EVAL_pitch_altered_v00.json',
    'hopkins_moore_2009': 'EVAL_hopkins_moore_2009.json',
    'spkr_discrimination_timit_ssn': 'EVAL_spkr_discrimination_timit_ssn.json',
}

EXPERIMENT_DATAFRAMES = util_spkr_word_psychophysics.run_spkr_word_experiments(
    list_regex_dir_model,
    dict_basename_eval=dict_basename_eval)

for tag_experiment in EXPERIMENT_DATAFRAMES.keys():
    df = EXPERIMENT_DATAFRAMES[tag_experiment]
    print(f"--> {tag_experiment} ({len(df)}) <--")
#     for c in df.columns:
#         print(f"|__ {c}")


In [ ]:
fn_data = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/model_data_spkr_word.pkl'
with open(fn_data, 'wb') as f:
    pickle.dump(EXPERIMENT_DATAFRAMES, f)

with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)

print(fn_data)


In [ ]:
importlib.reload(util_spkr_word_psychophysics)

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    # 'spearmanr': scipy.stats.spearmanr,
    'rmse': rmse,
}

experiment_to_compfunc_map = {
    'kell_like_word': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_kell_like,
        restrict_conditions=[0, 1, 2, 3]),
    'kell_like_word_dip_listening': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_kell_like,
        restrict_conditions=[3, 4]),
    'speech_in_synthetic_textures': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_speech_in_synthetic_textures),
    'pitch_altered_spkr_word': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task=['spkr', 'word']),
    'pitch_altered_word': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task='word'),
    'pitch_altered_spkr': functools.partial(
        util_spkr_word_psychophysics.compare_recognition_pitch_altered, key_task='spkr'),
    'hopkins_moore_2009_word': functools.partial(
        util_spkr_word_psychophysics.compare_word_recognition_hopkins_moore_2009),
    'spkr_discrimination_timit_ssn': functools.partial(
        util_spkr_word_psychophysics.compare_spkr_discrimination_timit_ssn),
}


def func_to_parallelize(tag_expt):
    compfunc = experiment_to_compfunc_map[tag_expt]
    tag_expt_for_dataframe = tag_expt
    for suffix in ['_spkr', '_word', '_dip_listening']:
        tag_expt_for_dataframe = tag_expt_for_dataframe.replace(suffix, '')
    df = EXPERIMENT_DATAFRAMES[tag_expt_for_dataframe]
    df_results = None
    for metric_key, metric_function in dict_metric_function.items():
        df_results_tmp = compfunc(
            df,
            bootstrap_repeats=1000,
            metric_function=metric_function)
        df_results_tmp = df_results_tmp.rename(columns={
            'metric': metric_key,
            'bootstrap_list_metric': f'bootstrap_list_{metric_key}',
        })
        df_results_tmp['tag_expt'] = tag_expt
        if df_results is None:
            df_results = df_results_tmp
        else:
            df_results = df_results.merge(df_results_tmp, on=['tag_expt', 'tag_model'])
    return df_results


with multiprocessing.Pool(60) as pool:
    list_df_results = pool.map(func_to_parallelize, list(experiment_to_compfunc_map.keys()))
df_results = pd.concat(list_df_results)

fn_metrics = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/human_model_comparison_metrics_spkr_word.pkl'
df_results.to_pickle(fn_metrics)
print(fn_metrics)

dict_agg = {}
for k in dict_metric_function.keys():
    dict_agg[k] = 'mean'
    dict_agg[f'bootstrap_list_{k}'] = list
df_results_mean = df_results.groupby(['tag_model']).agg(dict_agg).reset_index()
for k in df_results_mean.columns:
    if 'bootstrap_list_' in k:
        df_results_mean[k] = df_results_mean[k].map(lambda _: np.array(list(_)).mean(axis=0))
df_results_mean['tag_expt'] = 'AVERAGE'
df_results_mean


In [ ]:
df = pd.concat([df_results, df_results_mean])
list_tag_expt = np.unique(df.tag_expt)
list_tag_model = np.unique(df.tag_model)[::-1]
list_tag_model = [_ for _ in list_tag_model if 'taskSW/spont0' in _]
fig, ax = make_plot_aggregate_measures(
    df,
    str_ylabel="Human-model similarity\n(PEARSON R with 95% CI)",
    list_tag_expt=list_tag_expt,
    list_tag_model=list_tag_model)


In [ ]:
importlib.reload(util_spkr_word_psychophysics_figures)

list_tag_model = np.unique(df_results.tag_model.values)
list_tag_model = ['human'] + list_regex_dir_model[:4]
list_tag_expt = [
    'kell_like_word',
    'kell_like_word_dip_listening',
    'speech_in_synthetic_textures',
    'hopkins_moore_2009',
    'pitch_shifted',
    'pitch_condition',
    'kell_like_spkr',
    'kell_like_spkr_dip_listening',
    'spkr_discrimination_timit_ssn',
]

map_tag_expt_to_plot_func = {
    'kell_like_word': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_kell_like_recognition,
        key_task='word',
        fontsize_legend=7,
        restrict_background_condition=[2, 0, 3, 1]),
    'kell_like_word_dip_listening': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_kell_like_recognition,
        key_task='word',
        fontsize_legend=12,
        restrict_background_condition=[4, 3]),
    'speech_in_synthetic_textures': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_speech_in_synthetic_textures),
    'hopkins_moore_2009': util_spkr_word_psychophysics_figures.make_plot_hopkins_moore_2009_tfs_benefit,
    'pitch_shifted': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_pitch_shifted_recognition,
        key_task=['word', 'spkr']),
    'pitch_condition': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_pitch_condition_recognition,
        key_task=['word', 'spkr']),
    'kell_like_spkr': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_kell_like_recognition,
        key_task='spkr',
        fontsize_legend=7,
        restrict_background_condition=[2, 0, 3, 1],
        kwargs_plot_update={'ls': '--'}),
    'kell_like_spkr_dip_listening': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_kell_like_recognition,
        key_task='spkr',
        fontsize_legend=12,
        restrict_background_condition=[4, 3],
        kwargs_plot_update={'ls': '--'}),
    'spkr_discrimination_timit_ssn': functools.partial(
        util_spkr_word_psychophysics_figures.make_plot_voice_discrimination,
        kwargs_plot_update={'ls': '--'})
}

nrows = len(list_tag_expt)
ncols = len(list_tag_model)
figsize = (3.5 * ncols, 3 * nrows)
fig, ax_arr = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
ax_arr = ax_arr.reshape([nrows, ncols])

for r, tag_expt in enumerate(list_tag_expt):
    tag_expt_for_dataframe = tag_expt
    if 'pitch' in tag_expt:
        tag_expt_for_dataframe = 'pitch_altered'
    for suffix in ['_spkr', '_word', '_dip_listening']:
        tag_expt_for_dataframe = tag_expt_for_dataframe.replace(suffix, '')
    df = EXPERIMENT_DATAFRAMES[tag_expt_for_dataframe]
    for c, tag_model in enumerate(list_tag_model):
        ax = ax_arr[r, c]
        color, _ = get_color_and_label_from_model_tag(tag_model)
        map_tag_expt_to_plot_func[tag_expt](ax, df[df.tag_model == tag_model], color=color)
        if c > 0:
            ax.xaxis.label.set_color('w')
            ax.yaxis.label.set_color('w')
plt.tight_layout()
plt.show()


In [ ]:
importlib.reload(util_spkr_word_psychophysics_figures)

tag_expt = 'spkr_discrimination_timit_ssn'
list_tag_model = ['human'] + list_regex_dir_model
fig, ax = plt.subplots()
for tag_model in list_tag_model:
    df = EXPERIMENT_DATAFRAMES[tag_expt]
    df = df[df.tag_model == tag_model]
    color, _ = get_color_and_label_from_model_tag(tag_model)
    util_spkr_word_psychophysics_figures.make_plot_voice_discrimination(
        ax,
        df[df.tag_model == tag_model],
        color=color,
        include_legend=False)
plt.show()


In [ ]:
importlib.reload(util_localization_psychophysics)
list_regex_dir_model = [
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/grouped/before_pool_factor2/IHC3000Hz_anf384H160M096L/archFrancl??',    
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/IHC3000Hz_anf384H160M096L/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/IHC1000Hz_anf384H160M096L/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/IHC0320Hz_anf384H160M096L/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/IHC0050Hz_anf384H160M096L/archFrancl??',
    
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/grouped/before_pool_factor2/spont0_simplified_IHC3000/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/spont0_simplified_IHC3000/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/spont0_simplified_IHC1000/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/spont0_simplified_IHC0320/archFrancl??',
    '/om2/user/msaddler/tfauditoryutil/saved_models/models_localize/v01/spont0_simplified_IHC0050/archFrancl??',
]

# EXPERIMENT_DATAFRAMES = util_localization_psychophysics.run_localization_experiments(
#     list_regex_dir_model,
#     func_label_to_azim_elev=util_localization_psychophysics.label_to_azim_elev,
#     key_pred_prob='label_loc_int:probs_out',
#     key_pred='label_loc_int:labels_pred',
#     key_true='label_loc_int:labels_true',
#     n_loc_classes=504,
#     workers=60,
#     tag_ckpt='',
#     # list_expt=['speech_in_noise_in_reverb'],
# )

fn_data = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/model_data_localization.pkl'

# with open(fn_data, 'wb') as f:
#     pickle.dump(EXPERIMENT_DATAFRAMES, f)
# print(fn_data)

with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)


In [ ]:
importlib.reload(util_localization_psychophysics)

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    # 'spearmanr': scipy.stats.spearmanr,
    'rmse': rmse,
}

df_results = []
for tag_expt in tqdm.tqdm(EXPERIMENT_DATAFRAMES.keys()):
    df = EXPERIMENT_DATAFRAMES[tag_expt]
    df_results_tmp = util_localization_psychophysics.compare_localization_experiment(
        df,
        tag_expt=tag_expt,
        dict_metric_function=dict_metric_function,
        bootstrap_repeats=1000,
        workers=60)
    if df_results_tmp is not None:
        df_results.append(df_results_tmp)
df_results = pd.concat(df_results)

fn_metrics = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/human_model_comparison_metrics_localization.pkl'
df_results.to_pickle(fn_metrics)
print(fn_metrics)

dict_agg = {}
for k in dict_metric_function.keys():
    dict_agg[k] = 'mean'
    dict_agg[f'bootstrap_list_{k}'] = list
df_results_mean = df_results.groupby(['tag_model']).agg(dict_agg).reset_index()
for k in df_results_mean.columns:
    if 'bootstrap_list_' in k:
        df_results_mean[k] = df_results_mean[k].map(lambda _: np.array(list(_)).mean(axis=0))
df_results_mean['tag_expt'] = 'AVERAGE'
df_results_mean


In [ ]:
df = pd.concat([df_results, df_results_mean])
list_tag_expt = [
    'AVERAGE',
#     'snr_dependency',
#     'minimum_audible_angle',
#     'itd_ild_weighting',
#     'itd_threshold',
#     'new_ears',
#     'spectral_smoothing',
#     'precedence_effect_localization',
#     'mp_spectral_cues',
#     'bandwidth_dependency',
]
# list_tag_expt = np.unique(df.tag_expt)
list_tag_model = np.unique(df.tag_model)[::-1]
# list_tag_model = [
#     _ for _ in list_tag_model if ('3000' in _) or ('pool' in _)
# ]
fig, ax = make_plot_aggregate_measures(
    df,
    str_ylabel="Human-model similarity\n(PEARSON R with 95% CI)",
    list_tag_expt=list_tag_expt,
    list_tag_model=list_tag_model,
    width=0.05)


In [ ]:
import util_pitchnet_psychophysics_figures
importlib.reload(util_pitchnet_psychophysics_figures)

list_regex_dir_model = [
    'human',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/IHC3000Hz_anf192H080M048L/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/IHC1000Hz_anf192H080M048L/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/TRANSPOSED_IHC0320Hz_anf019H008M005L/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/TRANSPOSED_IHC0050Hz_anf019H008M005L/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/spont0_simplified_IHC3000/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/spont0_simplified_IHC1000/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/spont0_simplified_IHC0320_TRANSPOSED/arch_0???',
    '/om2/user/msaddler/tfauditoryutil/saved_models/PNDv08/spont0_simplified_IHC0050_TRANSPOSED/arch_0???',
]

experiment_to_func = {
    'bernox2005': util_pitchnet_psychophysics_figures.load_data_bernox2005,
    'mcpherson_snr': util_pitchnet_psychophysics_figures.load_data_mcpherson_snr,
    'transposedtones': util_pitchnet_psychophysics_figures.load_data_transposedtones,
    'pure_tone_spl': util_pitchnet_psychophysics_figures.load_data_pure_tone_spl,
    'freqshiftedcomplexes': util_pitchnet_psychophysics_figures.load_data_freqshiftedcomplexes,
    'mistunedharmonics': util_pitchnet_psychophysics_figures.load_data_mistunedharmonics,
    'altphasecomplexes': lambda _: util_pitchnet_psychophysics_figures.load_data_altphasecomplexes(_, bin_step=0.04),
}

EXPERIMENT_DATAFRAMES = {}
for tag_expt in experiment_to_func.keys():
    print(tag_expt)
    EXPERIMENT_DATAFRAMES[tag_expt] = experiment_to_func[tag_expt](list_regex_dir_model)

fn_data = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/model_data_pitchnet.pkl'
with open(fn_data, 'wb') as f:
    pickle.dump(EXPERIMENT_DATAFRAMES, f)
with open(fn_data, 'rb') as f:
    EXPERIMENT_DATAFRAMES = pickle.load(f)
print(fn_data)


In [ ]:
importlib.reload(util_pitchnet_psychophysics_figures)

def rmse(x, y):
    return np.linalg.norm(x - y) / np.sqrt(x.shape[0])

dict_metric_function = {
    'pearsonr': scipy.stats.pearsonr,
    # 'spearmanr': scipy.stats.spearmanr,
    'rmse': rmse,
}

df_results = []
for tag_expt in tqdm.tqdm(EXPERIMENT_DATAFRAMES.keys()):
    df = EXPERIMENT_DATAFRAMES[tag_expt]
    df_results_tmp = util_pitchnet_psychophysics_figures.compare_pitchnet_experiment(
        df,
        tag_expt=tag_expt,
        dict_metric_function=dict_metric_function,
        workers=40)
    df_results.append(df_results_tmp)
df_results = pd.concat(df_results)

fn_metrics = '/om2/user/msaddler/tfauditoryutil/paper_phaselocknet/data/human_model_comparison_metrics_pitchnet.pkl'
df_results.to_pickle(fn_metrics)
print(fn_metrics)

dict_agg = {}
for k in dict_metric_function.keys():
    dict_agg[k] = 'mean'
    dict_agg[f'bootstrap_list_{k}'] = list
df_results_mean = df_results.groupby(['tag_model']).agg(dict_agg).reset_index()
for k in df_results_mean.columns:
    if 'bootstrap_list_' in k:
        df_results_mean[k] = df_results_mean[k].map(lambda _: np.array(list(_)).mean(axis=0))
df_results_mean['tag_expt'] = 'AVERAGE'
df_results_mean


In [ ]:
df = pd.concat([df_results, df_results_mean])
list_tag_expt = ['AVERAGE'] + list(EXPERIMENT_DATAFRAMES.keys())
list_tag_model = np.unique(df['tag_model'].values)[::-1]
# list_tag_model = [list_tag_model[_] for _ in [2,3,0,1]]

for tag_model in list_tag_model:
    print(tag_model)

fig, ax = make_plot_aggregate_measures(
    df,
    str_ylabel="Human-model similarity\n(PEARSON R with 95% CI)",
    width=0.15,
#     key_metric='mae',
#     str_ylabel=None,
#     ylimits=None,
#     yticks=None,
    list_tag_expt=list_tag_expt,
    list_tag_model=list_tag_model)
